In [7]:
%pip install --upgrade pip
# install dependencies and select solver
%pip install -q amplpy matplotlib pandas

SOLVER = "cplex"

from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["cplex"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


In [16]:
import json
import numpy as np
import random
from itertools import combinations

m = AMPL()
m.read("objectives.mod")

m.option["solver"] = SOLVER

sats = []
NUM_SATS = 6
for i in range(NUM_SATS):
    sats.append(i+1)

timesteps = []
for i in range(1000):
    timesteps.append(i+1)

# m.set["SATS"] = list(sats.keys()) 
m.set["SATS_1"] = sats
m.set["SATS_2"] = sats

# m.set["TIMESTEPS"] = list(timesteps.keys())
m.set["TIMESTEPS"] = timesteps

m.param["sat_max"] = len(sats)
m.param["timestep_max"] = len(timesteps)
# m.param["cost"] = {sat: cost for sat, (cost, _, _) in sats.items()}
# m.param["f_min"] = {sat: f_min for sat, (_, f_min, _) in sats.items()}
# m.param["f_max"] = {sat: f_max for sat, (_, _, f_max) in sats.items()}

# m.param["n_min"] = {time: n_min for time, (n_min, _) in timesteps.items()}
# m.param["n_max"] = {time: n_max for time, (_, n_max) in timesteps.items()}

# Amounts is given as the sat at the timestep value 2D array[SATS, TIMESTEP]
connections = []
connections = np.empty((len(sats), len(sats)), dtype=object)
for j in range(len(sats)):
    for k in range(len(sats)):
        temp = []
        if j!=k:
            for i in range(len(timesteps)):
                choice = random.randint(0,1)
                if choice == 1:
                    temp.append(i+1)
                else:
                    temp.append(99999999)
        else:
            for i in range(len(timesteps)):
                temp.append(len(timesteps)+1)
        connections[j,k] = np.array(temp)

m.param["conns"] = {
    (sat1, sat2, time): connections[j,k][i]
    for i, time in enumerate(timesteps)
    for j, sat1 in enumerate(sats)
    for k, sat2 in enumerate(sats)
}

items = np.array(np.linspace(1, len(sats), len(sats)), dtype=int)
comb = combinations(items, 3) 
comb = list(comb)
comb_all = []
for i in range(1, len(sats)):
    for co in comb:
        if i not in co:
            comb_all.append(np.append([i],co))
comb = np.array(comb_all)

m.param["combs_num"] = len(comb)



m.param["c"] = {
    (i+1, j+1): comb[i][j]
    for i in range(np.shape(comb)[0])
    for j in range(np.shape(comb)[1])
}

# m.eval("display combinations;")

# m.eval("display consensus_time1234_1;")

m.solve()
total_cost = m.get_objective("Total_Cost")
print("Objective is:", total_cost.get().value())
sats_result = np.array(m.getVariable("sats").get_values().to_list())
print(sats_result[:,1])

CPLEX 22.1.1.0: optimal integer solution; objective 17
0 MIP simplex iterations
0 branch-and-bound nodes
Objective is: 17.0
[1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
